In [1]:
import requests
import nba_api
import pandas as pd
import numpy as np
from datetime import date
from nba_api.stats.endpoints import (playbyplayv2,playbyplay, leaguehustlestatsplayer,
                                     leaguedashptstats,leaguestandings,
                                     playerdashptreb,leaguedashplayerbiostats,
                                    leaguedashplayerptshot)
import time

In [2]:
current_year="2023-24"

In [3]:
pbp2023=pd.read_csv(filepath_or_buffer="Data/"+current_year+"_pbp.csv")

# The Spark Plug Award (sponsored by Lt. Surge, presented by American Express CEO Stephen J Squeri)
Most charges drawn per 36 minutes (minimum 70% of games played), credit to morron88 for the idea to separate charges & loose balls in 2020

In [4]:
compact_standings=leaguestandings.LeagueStandings(league_id='00',
                                             season=current_year,season_type="Regular Season").\
standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']]
compact_standings['TeamGP']=compact_standings.WINS+compact_standings.LOSSES

hustle=leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="PerGame",season=current_year,
    season_type_all_star="Regular Season").hustle_stats_player.get_data_frame()

In [5]:
hustle_w_gp_qualify=hustle.merge(compact_standings,how='left',left_on='TEAM_ID',right_on='TeamID')
hustle_w_gp_qualify['G_PERCENT']=hustle_w_gp_qualify.G/hustle_w_gp_qualify.TeamGP
hustle_70percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.7').copy()
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_70percent_gp.MIN,axis=0).multiply(36,axis=0)

In [6]:
hustle_70percent_gp.nlargest(10,columns='CHARGES_DRAWN',keep='all')[['PLAYER_NAME','CHARGES_DRAWN']]

,PLAYER_NAME,CHARGES_DRAWN
267,Jaylin Williams,0.775385
50,Brandin Podziemski,0.690226
415,Moritz Wagner,0.610169
179,Garrison Mathews,0.552000
491,Sam Merrill,0.473143
204,Isaiah Joe,0.467027
332,Kenrich Williams,0.459060
233,Jalen Brunson,0.457627
357,Kyle Lowry,0.408511
527,Tim Hardaway Jr.,0.362687


# The Most Loose Balls Recovered Award (sponsored by Hungry Hungry Hippos, presented by Dennis Rodman & [Nene’s doctor](https://www.espn.com/nba/news/story?id=3197423))

Per 36 minutes, minimum 70% of games played

In [7]:
hustle_70percent_gp.nlargest(10,columns='LOOSE_BALLS_RECOVERED',keep='all')[['PLAYER_NAME','LOOSE_BALLS_RECOVERED']]

,PLAYER_NAME,LOOSE_BALLS_RECOVERED
306,Josh Okogie,1.732500
377,Malachi Flynn,1.530709
463,Paul Reed,1.521649
511,T.J. McConnell,1.483516
537,Trendon Watford,1.482353
417,Moses Moody,1.378286
228,Jae'Sean Tate,1.358491
13,Alex Caruso,1.291986
485,Russell Westbrook,1.264000
324,Kawhi Leonard,1.248980


# The Plexiglass Award

most deflections per 36 minutes, minimum 70% of games played

In [8]:
hustle_70percent_gp.nlargest(10,columns='DEFLECTIONS',keep='all')[['PLAYER_NAME','DEFLECTIONS']]

,PLAYER_NAME,DEFLECTIONS
398,Matisse Thybulle,5.313537
13,Alex Caruso,4.678746
164,Dyson Daniels,4.520179
352,Kris Dunn,4.190476
511,T.J. McConnell,3.956044
463,Paul Reed,3.804124
501,Shai Gilgeous-Alexander,3.716471
153,Donte DiVincenzo,3.686598
125,De'Aaron Fox,3.650139
328,Kelly Olynyk,3.584000


# The Wes Unseld Memorial Brick Wall Award

most points generated by screen assists per 36 minutes, minimum 70% of games played

In [9]:
hustle_70percent_gp.nlargest(10,columns='SCREEN_AST_PTS',keep='all')[['PLAYER_NAME','SCREEN_AST_PTS']]

,PLAYER_NAME,SCREEN_AST_PTS
342,Kevon Looney,15.160248
252,James Wiseman,14.441618
150,Domantas Sabonis,14.137815
162,Dwight Powell,13.669173
430,Nick Richards,13.479389
534,Trayce Jackson-Davis,13.207229
569,Zeke Nnaji,12.981818
320,Jusuf Nurkic,12.870330
235,Jalen Duren,12.841237
124,Day'Ron Sharpe,12.182781


# The “He Trick Y’All, Running Around, Doing Nothing” Award (sponsored by Russell Westbrook, presented by Tony Snell)*

Lowest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played)

In [10]:
hustle_50percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.5').copy()
#traditional defensive stats approximate by tracking
#(deflections ~ steals, contested shots ~ blocks, def reb ~ boxouts)
per_36_percent_ranks=hustle_50percent_gp[['CHARGES_DRAWN', 'CONTESTED_SHOTS_2PT','CONTESTED_SHOTS_3PT', 'DEFLECTIONS', 
               'DEF_BOXOUTS','DEF_LOOSE_BALLS_RECOVERED']].\
div(hustle_50percent_gp.MIN,axis=0).multiply(36,axis=0).apply(lambda x: x.rank(pct=True)).add_suffix("_pct_rank")
per_36_percent_ranks["sum"]=per_36_percent_ranks.sum(axis=1)
hustle_50percent_gp_ranks=hustle_50percent_gp.merge(per_36_percent_ranks,how='left',left_index=True,right_index=True)

In [11]:
hustle_50percent_gp_ranks.to_csv(path_or_buf="Output Data/Hustle Ranks.csv")

In [12]:
hustle_50percent_gp_ranks.nsmallest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
378,Malaki Branham,0.189655,0.149425,0.155172,0.025862,0.060345,0.359195,0.939655
406,Michael Porter Jr.,0.189655,0.431034,0.112069,0.097701,0.192529,0.020115,1.043103
348,Klay Thompson,0.189655,0.367816,0.241379,0.114943,0.206897,0.077586,1.198276
510,Svi Mykhailiuk,0.189655,0.109195,0.140805,0.497126,0.316092,0.057471,1.310345
387,Marcus Morris Sr.,0.189655,0.224138,0.031609,0.071839,0.724138,0.094828,1.336207
519,Terance Mann,0.439655,0.284483,0.037356,0.135057,0.339080,0.114943,1.350575
26,Anfernee Simons,0.472701,0.137931,0.183908,0.034483,0.008621,0.548851,1.386494
489,Saddiq Bey,0.189655,0.278736,0.043103,0.229885,0.301724,0.402299,1.445402
438,Norman Powell,0.189655,0.160920,0.198276,0.330460,0.344828,0.227011,1.451149
176,GG Jackson,0.189655,0.468391,0.120690,0.103448,0.399425,0.175287,1.456897


In [13]:
hustle_50percent_gp_ranks[hustle_50percent_gp_ranks['PLAYER_NAME'].isin(['Patrick Beverley','Russell Westbrook'])].filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
459,Patrick Beverley,0.869253,0.158046,0.385057,0.656609,0.020115,0.816092,2.905172
485,Russell Westbrook,0.804598,0.126437,0.045977,0.844828,0.038793,0.945402,2.806034


# The "Got that Dawg in Him" Award (presented by Air Bud)*

Highest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played) (credit to memeticengineering for the idea)

In [14]:
hustle_50percent_gp_ranks.nlargest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
267,Jaylin Williams,1.000000,0.919540,0.997126,0.531609,0.943966,0.827586,5.219828
228,Jae'Sean Tate,0.959770,0.658046,0.982759,0.795977,0.787356,0.968391,5.152299
168,Eugene Omoruyi,0.962644,0.712644,0.856322,0.971264,0.744253,0.732759,4.979885
162,Dwight Powell,0.853448,0.971264,0.813218,0.729885,0.965517,0.633621,4.966954
21,Andre Drummond,0.692529,0.813218,0.936782,0.922414,0.781609,0.744253,4.890805
90,Cody Zeller,0.991379,0.890805,0.485632,0.560345,0.997126,0.895115,4.820402
156,Draymond Green,0.985632,0.844828,0.991379,0.818966,0.925287,0.244253,4.810345
197,Herbert Jones,0.910920,0.534483,0.974138,0.870690,0.715517,0.672414,4.678161
463,Paul Reed,0.189655,0.945402,0.741379,0.962644,0.885057,0.908046,4.632184
110,Daniel Gafford,0.793103,0.974138,0.482759,0.620690,0.962644,0.770115,4.603448


# The Trickshot Grenadier Award (presented by Dude Perfect)

Highest sum of percentile ranks in FGA, FGA frequency & eFG% on shots with 4 seconds or less on the shotclock (minimum 50th percentile in FGA) (credit to BehavioralSink & Bylanta for the idea)

In [15]:
late_shotclock_shots=leaguedashplayerptshot.LeagueDashPlayerPtShot(shot_clock_range_nullable="4-0 Very Late").league_dash_ptshots.get_data_frame().loc[:, 'PLAYER_ID':'EFG_PCT']
late_shotclock_shots["FGA_percentile"]=late_shotclock_shots.FGA.rank(method='min',pct=True)
late_shotclock_shots_top_50_percentile=late_shotclock_shots.query("FGA_percentile>0.5").copy()
late_shotclock_shots_top_50_percentile["FGA_FREQ_percentile"]=late_shotclock_shots_top_50_percentile.FGA_FREQUENCY.rank(method='min',pct=True)
late_shotclock_shots_top_50_percentile["EFG_percentile"]=late_shotclock_shots_top_50_percentile.EFG_PCT.rank(method='min',pct=True)
late_shotclock_shots_top_50_percentile["sum_percentiles"]=late_shotclock_shots_top_50_percentile[['FGA_FREQ_percentile','FGA_percentile','EFG_percentile']].sum(axis=1)
late_shotclock_shots_top_50_percentile.to_csv(path_or_buf="Output Data/Trickshot Grenadier.csv",index=False)
late_shotclock_shots_top_50_percentile.nlargest(n=10,columns="sum_percentiles")

,PLAYER_ID,PLAYER_NAME,PLAYER_LAST_TEAM_ID,PLAYER_LAST_TEAM_ABBREVIATION,AGE,GP,G,FGA_FREQUENCY,FGM,FGA,FG_PCT,EFG_PCT,FGA_percentile,FGA_FREQ_percentile,EFG_percentile,sum_percentiles
6,202695,Kawhi Leonard,1610612746,LAC,32.0,67,58,0.116,59,132,0.447,0.519,0.986817,0.804511,0.872180,2.663509
28,1629636,Darius Garland,1610612739,CLE,24.0,57,47,0.127,44,107,0.411,0.509,0.947269,0.853383,0.819549,2.620202
0,203999,Nikola Jokic,1610612743,DEN,29.0,76,67,0.131,78,179,0.436,0.475,1.000000,0.872180,0.714286,2.586466
37,202699,Tobias Harris,1610612755,PHI,31.0,69,45,0.106,47,99,0.475,0.540,0.928437,0.687970,0.906015,2.522422
12,1630532,Franz Wagner,1610612753,ORL,22.0,71,55,0.110,51,118,0.432,0.496,0.975518,0.751880,0.781955,2.509352
254,1631367,Jacob Gilyard,1610612751,BKN,25.0,40,20,0.185,12,27,0.444,0.630,0.512241,0.996241,0.977444,2.485925
121,1627936,Alex Caruso,1610612741,CHI,30.0,69,36,0.107,28,56,0.500,0.679,0.766478,0.706767,0.992481,2.465726
218,204060,Joe Ingles,1610612753,ORL,36.0,67,27,0.146,14,33,0.424,0.576,0.581921,0.932331,0.943609,2.457861
79,1629130,Duncan Robinson,1610612748,MIA,29.0,67,47,0.106,29,71,0.408,0.556,0.847458,0.687970,0.921053,2.456480
20,202710,Jimmy Butler,1610612748,MIA,34.0,59,50,0.143,44,112,0.393,0.442,0.958569,0.917293,0.563910,2.439772


# The "David vs Goliath" Award (presented by Dwyane Wade)*

most shots blocked where the blocker is at least 5 inches shorter than the blockee

In [16]:
blocks=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('BLOCK',na=False)|
                          pbp2023['VISITORDESCRIPTION'].str.contains('BLOCK',na=False)]

player_bio=leaguedashplayerbiostats.LeagueDashPlayerBioStats().league_dash_player_bio_stats.get_data_frame()

blocking_player_heights=blocks.merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER1_ID',right_on='PLAYER_ID').merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER3_ID',right_on='PLAYER_ID')
blocking_player_heights["height_difference"]=blocking_player_heights['PLAYER_HEIGHT_INCHES_x']-blocking_player_heights['PLAYER_HEIGHT_INCHES_y']
blocking_player_heights.query('height_difference >= 5').groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
81,Fred VanVleet,41
68,Derrick White,26
164,Luguentz Dort,20
31,CJ McCollum,17
158,Kyrie Irving,17
75,Donovan Mitchell,16
51,D'Angelo Russell,14
87,Grayson Allen,13
112,Jalen Williams,12
115,James Harden,12


# The "Pick On Someone Your Own Size" Award

most shots blocked where the blocker is at least 5 inches taller than the blockee

In [17]:
blocking_player_heights.query('height_difference <= -5').groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
249,Victor Wembanyama,221
26,Brook Lopez,124
36,Chet Holmgren,122
222,Rudy Gobert,98
251,Walker Kessler,87
157,Kristaps Porzingis,85
193,Nic Claxton,75
11,Anthony Davis,65
66,Dereck Lively II,61
189,Myles Turner,57


# The “Fine, I’ll Do It Myself” Award (sponsored by Thanos, presented by Allen Iverson)

Highest percentage of unassisted field goals, minimum 50% of games played (https://www.nba.com/stats/players/scoring/?sort=GP&dir=-1)

1. Luka Doncic (78%)
2. Shai Gilgeous-Alexander (76%)
3. Trae Young (71.4%)
4. James Harden (68.2%)
5. Damian Lillard (67.7%)

# The “You Gotta Feed Me” Award (presented by Joey Chestnut & Marcin Gortat)

Highest percentage of assisted field goals, minimum 50% of games played

1. Reggie Bullock (97.1%)
2. Doug McDermott (96.7%)
3. Davis Bertans (95.7%)
4. Sam Merrill (94.9%)
5. Caleb Houstan (93.8%)

# The “FUCK OUTTA HERE, I GOT THAT SHIT” Award (presented by Carmelo Anthony)

Lowest contested rebound percentage, minimum 50% of games played

In [18]:
rebounding=playerdashptreb.PlayerDashPtReb(team_id=0,player_id=0).overall_rebounding.get_data_frame()

games_percentages=hustle_w_gp_qualify.copy()[['PLAYER_ID','PLAYER_NAME','G_PERCENT']]

reb_w_gp_qualify=rebounding.merge(games_percentages,left_on='PLAYER_ID',right_on='PLAYER_ID').query('G_PERCENT >= 0.5')

reb_w_gp_qualify.nsmallest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
467,Marcus Sasser,0.060
100,Tyus Jones,0.061
68,Reggie Bullock Jr.,0.076
219,Trae Young,0.077
254,Tyler Herro,0.086
524,Nick Smith Jr.,0.092
271,Jordan Poole,0.094
512,Keyonte George,0.095
312,Immanuel Quickley,0.100
188,Jevon Carter,0.102


alternatively: restricting to players > 6 foot 6 inches in height

In [19]:
above_66=player_bio.query('PLAYER_HEIGHT_INCHES > 6*12+6')

above_66.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nsmallest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
154,Sam Hauser,0.180
87,Joe Ingles,0.181
80,Jayson Tatum,0.185
5,Amir Coffey,0.191
48,Duncan Robinson,0.194
73,Jalen McDaniels,0.194
105,Kevin Huerter,0.199
23,Cameron Johnson,0.200
160,Svi Mykhailiuk,0.200
166,Trey Murphy III,0.201


# The "Glass Cleaner" Award (presented by Dennis Rodman, sponsored by Windex)

Highest contested rebound percentage, minimum 50% of games played (https://www.nba.com/stats/players/rebounding?PerMode=Totals&dir=D&sort=REB_CONTEST_PCT)

In [20]:
reb_w_gp_qualify.nlargest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
311,Zeke Nnaji,0.557
173,Luke Kornet,0.538
368,Day'Ron Sharpe,0.534
364,Isaiah Jackson,0.530
313,Paul Reed,0.521
16,JaVale McGee,0.504
448,Walker Kessler,0.501
61,Cody Zeller,0.495
164,Isaiah Hartenstein,0.493
475,Trayce Jackson-Davis,0.485


alternatively: restricting to players < 6 foot 7 inches in height

In [21]:
below_67=player_bio.query('PLAYER_HEIGHT_INCHES < 6*12+7')
below_67.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nlargest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
61,Eugene Omoruyi,0.481
101,Josh Okogie,0.453
108,Kenrich Williams,0.410
125,Markelle Fultz,0.404
78,Jaden Springer,0.382
171,Zion Williamson,0.378
4,Aaron Wiggins,0.374
138,Ochai Agbaji,0.373
80,Jae'Sean Tate,0.371
68,Grant Williams,0.366
